In [ ]:
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score

In [ ]:
df = pd.read_excel('Vehicle_Crashes_in_Iowa_20250410.xlsx')
display(df)
print(df.info())

In [ ]:
# Capping property damage at 1st and 99th percentiles to prevent skewing
low, high = df['Amount of Property Damage'].quantile([0.05, 0.95])
df['Damage_capped'] = df['Amount of Property Damage'].clip(low, high)

# Log-transforming the capped damage for analysis
df['Damage_log'] = np.log1p(df['Damage_capped'])

In [ ]:
# Creating feature interactions
df['Light_x_Weather']          = df['Light Conditions'] + ':' + df['Weather Conditions']
df['MajorCause_x_Surface']     = df['Major Cause']     + ':' + df['Surface Conditions']
df['Route_x_Event']            = df['Route with System'] + ':' + df['First Harmful Event']
df['WorkZone_x_District']      = df['Work Zone']       + ':' + df['DOT District']
df['Month_x_Day']              = df['Month of Crash']  + ':' + df['Day of Week']

In [ ]:
# Defining the features and targets for analysis
features = [
    'Month of Crash', 'Day of Week', 'Hour', 'DOT District', 'City Name', 'County Name',
    'Route with System', 'First Harmful Event', 'Location of First Harmful Event',
    'Manner of Crash/Collision', 'Major Cause', 'Drug or Alcohol ', 'Environmental Conditions',
    'Light Conditions', 'Surface Conditions', 'Weather Conditions', 'Roadway Contribution',
    'Roadway Type', 'Roadway Surface', 'Work Zone', 'Total Number of Occupants',
    'Light_x_Weather', 'MajorCause_x_Surface', 'Route_x_Event',
    'WorkZone_x_District', 'Month_x_Day'
]
target_casualties = ['Number of Fatalities', 'Number of Injuries']
target_damage_log = 'Damage_log'
target_damage_orig = 'Damage_capped'

# Converting categorical columns to string to resolve type issues with OneHotEncoder
for col in categorical_features:
    df[col] = df[col].astype(str)
    
numeric_features = ['Total Number of Occupants']
categorical_features = [col for col in features if col not in numeric_features]

In [ ]:
# Preparing training/testing data split
X = df[features]
y_cas = df[target_casualties]
y_dmg_log = df[target_damage_log]
y_dmg_orig = df[target_damage_orig]

X_train, X_test, y_train_cas, y_test_cas, y_train_dmg_log, y_test_dmg_log, y_train_dmg_orig, y_test_dmg_orig = train_test_split(
    X, y_cas, y_dmg_log, y_dmg_orig, test_size=0.2, random_state=42
)

In [ ]:
# Preprocessing pipeline
preprocessor = ColumnTransformer([
    ('num', SimpleImputer(strategy='median'), numeric_features),
    ('cat', Pipeline([
        ('impute', SimpleImputer(strategy='constant', fill_value='Missing')),
        ('ord', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
    ]), categorical_features)
])

# Casualties pipeline
hgb_cas = HistGradientBoostingRegressor(random_state=42, max_iter=100, early_stopping='auto')
cas_pipeline = Pipeline([
    ('pre', preprocessor),
    ('model', MultiOutputRegressor(hgb_cas))
])
cas_pipeline.fit(X_train, y_train_cas)
y_test_cas_pred = cas_pipeline.predict(X_test)

# Damage pipeline
hgb_dmg = HistGradientBoostingRegressor(
    loss='quantile',
    quantile=0.5,
    random_state=42,
    max_iter=100,
    early_stopping='auto'
)
dmg_pipeline = Pipeline([
    ('pre', preprocessor),
    ('model', hgb_dmg)
])
dmg_pipeline.fit(X_train, y_train_dmg_log)
y_test_dmg_log_pred = dmg_pipeline.predict(X_test)

# Back-transforming predictions to original scale
y_test_dmg_pred = np.expm1(y_test_dmg_log_pred)

In [ ]:
# Evaluating and collecting metrics
rows = []

# Casualties metrics
for idx, target in enumerate(target_casualties):
    rows.append({
        'target': target,
        'dataset': 'test',
        'RMSE': root_mean_squared_error(y_test_cas[target], y_test_cas_pred[:, idx]),
        'MAE': mean_absolute_error(y_test_cas[target], y_test_cas_pred[:, idx]),
        'R2': r2_score(y_test_cas[target], y_test_cas_pred[:, idx])
    })

# Damage metrics (original scale)
rows.append({
    'target': 'Amount of Property Damage',
    'dataset': 'test',
    'RMSE': root_mean_squared_error(y_test_dmg_orig, y_test_dmg_pred),
    'MAE': mean_absolute_error(y_test_dmg_orig, y_test_dmg_pred),
    'R2': r2_score(y_test_dmg_orig, y_test_dmg_pred)
})

metrics_df = pd.DataFrame(rows)
display(metrics_df)

In [ ]:
metrics_df.to_excel('metrics_output.xlsx', index=False)